# Group Delay Extraction
A 2DSI map contains the information of the laser pulses' spectral phase. This information is directly retrievable, which is a big advantage compared to other methods.
Once a 2DSI map is experimentally measured, the phase retrieval algorithm is applied. In this Notebook the 2DSI map's data, which was created computationally in the 2DSI Map Creator Notebook, will be taken for retrieval.

The extraction of the Group Delay (GD) is the first step of the phase retrieval algorithm.

$$S_{2DSI}(\omega, \tau_{CW}) = S(\omega) + S(\omega-\Omega) + \sqrt{S(\omega) S(\omega-\Omega)} cos(\Phi_0 + \color{green}{\Phi(\omega)- \Phi(\omega - \Omega)})$$

$S_{2DSI}(\omega, \tau_{CW})$ is describing the behaviour of it and the relevant information lies within the argument of the cosine.

This is because $$GD(\omega) = \frac{d\Phi}{d\omega} \approx \frac{\color{green}{\Phi(\omega)- \Phi(\omega - \Omega)}}{\Omega}.$$

(More on how the phase will be exactly taken from the measured 2DSI map will be covered in the Group Delay Integration Notebook.)

## Import of the data:

In [9]:
#Import of packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import sys
from scipy.interpolate import interp1d

#Import of the data
df_pulse = pd.read_csv("Data/pulse.csv", index_col = 0)
df_2DSImap = pd.read_csv("Data/2DSImap.csv", index_col = 0)

In [70]:
#Create numpy arrays
#Pulse
f = df_pulse.loc[:, 'f (Hz)'].to_numpy()           #frequency axis pulse
i_f = df_pulse.loc[:, 'spec (norm.)'].to_numpy()   #spectrum (intensity) pulse
phase = df_pulse.loc[:, 'phase (rad)'].to_numpy()  #phase of the pulse

#2DSI map
map_2DSI = df_2DSImap.loc[:, :].to_numpy()         #2DSI map values

phi_0_array = df_2DSImap.index.to_numpy()          #y-axis
phi_0_array = phi_0_array.astype(float)            #cast elements to float

f_2DSI = df_2DSImap.columns.to_numpy()             #x-axis
f_2DSI = f_2DSI.astype(float)                      #cast elements to float